In [ ]:
!pip install datasets transformers
from datasets import load_dataset
from transformers import AutoTokenizer
import numpy as np
import pandas as pd
import multiprocessing as mp
import itertools
import pickle
from google.colab import files, drive

drive.mount('/content/gdrive')

#Settings
corpus = 'bookcorpus' #'openwebtext', 'bookcorpus'
model_name = 'xlnet-large-cased' #'bert-large-uncased'(='funnel-transformer_large'), 'roberta-large' 
data_dir = '/content/gdrive/MyDrive/corpus/'
steps = 100

# Load corpus dataset
corpus_dataset = load_dataset(corpus)

# Load tokenizer
tokenizer = AutoTokenizer.from_pretrained(model_name)

     |████████████████████████████████| 266kB 5.0MB/s 
     |████████████████████████████████| 2.5MB 48.5MB/s 
     |████████████████████████████████| 245kB 54.3MB/s 
     |████████████████████████████████| 122kB 53.6MB/s 
     |████████████████████████████████| 3.3MB 49.5MB/s 
     |████████████████████████████████| 901kB 50.5MB/s 
ERROR: transformers 4.8.2 has requirement huggingface-hub==0.0.12, but you'll have huggingface-hub 0.0.13 which is incompatible.
Mounted at /content/gdrive


Dataset bookcorpus downloaded and prepared to /root/.cache/huggingface/datasets/bookcorpus/plain_text/1.0.0/44662c4a114441c35200992bea923b170e6f13f2f0beb7c14e43759cec498700. Subsequent calls will reuse this data.


## Write to files

In [ ]:
def asr_checker(current_sentence):
  sentence_tokens = tokenizer.encode_plus(current_sentence, return_tensors = "pt")['input_ids'].numpy()[0].astype(int)
  sentence_tokens = sentence_tokens[1:len(sentence_tokens)-1]

  asrS = []

  if len(sentence_tokens) < 3: 
    return 0, None, sentence_tokens.tolist()

  for j in range(len(sentence_tokens)-2):
    if sentence_tokens[j] == sentence_tokens[j+1] and sentence_tokens[j] != sentence_tokens[j+2]:
      asrS.insert(len(asrS), [int(sentence_tokens[j]), int(sentence_tokens[j+1]), int(sentence_tokens[j+2])])
    elif sentence_tokens[j] == sentence_tokens[j+2] and sentence_tokens[j] != sentence_tokens[j+1]:
      asrS.insert(len(asrS), [int(sentence_tokens[j]), int(sentence_tokens[j+1]), int(sentence_tokens[j+2])])
    elif sentence_tokens[j+1] == sentence_tokens[j+2] and sentence_tokens[j] != sentence_tokens[j+1]:
      asrS.insert(len(asrS), [int(sentence_tokens[j]), int(sentence_tokens[j+1]), int(sentence_tokens[j+2])])
  
  if len(asrS)==0: return j, None, sentence_tokens.tolist()
  else: 
    return j, asrS, sentence_tokens.tolist()


leng = len(corpus_dataset['train'])
lims = leng//steps
rem = leng%steps
lims_list = list(range(0,lims*steps+1, lims))
last = lims_list[len(lims_list)-1] 
if rem != 0: lims_list.append(last+rem)

for i in range(len(lims_list)-1):
  pool = mp.Pool(mp.cpu_count())
  all_raw = pool.map(asr_checker, [sentence for sentence in corpus_dataset['train'][lims_list[i]:lims_list[i+1]]['text']])
  pool.close()
  
  with open(data_dir+str(lims_list.index(lims_list[i]))+'pickle.dat', 'wb') as f:
    pickle.dump(all_raw, f)

  print(f'Step: {lims_list.index(lims_list[i])}\nProcessed up to sentence: {lims_list[i+1]}')


Step: 0
Processed up to sentence: 740042
Step: 1
Processed up to sentence: 1480084
Step: 2
Processed up to sentence: 2220126
Step: 3
Processed up to sentence: 2960168
Step: 4
Processed up to sentence: 3700210
Step: 5
Processed up to sentence: 4440252
Step: 6
Processed up to sentence: 5180294
Step: 7
Processed up to sentence: 5920336
Step: 8
Processed up to sentence: 6660378
Step: 9
Processed up to sentence: 7400420
Step: 10
Processed up to sentence: 8140462
Step: 11
Processed up to sentence: 8880504
Step: 12
Processed up to sentence: 9620546
Step: 13
Processed up to sentence: 10360588
Step: 14
Processed up to sentence: 11100630
Step: 15
Processed up to sentence: 11840672
Step: 16
Processed up to sentence: 12580714
Step: 17
Processed up to sentence: 13320756
Step: 18
Processed up to sentence: 14060798
Step: 19
Processed up to sentence: 14800840
Step: 20
Processed up to sentence: 15540882
Step: 21
Processed up to sentence: 16280924
Step: 22
Processed up to sentence: 17020966
Step: 23
Pro

## Read data

In [ ]:
def loadall(filename):
  with open(filename, 'rb') as f:
    while True:
      try:
        yield pickle.load(f)
      except EOFError:
        break

"""Process ASR"""
asr_index_raw = [[],[]]
for i in range(len(lims_list)-1):
  asr_index_raw[0].extend([x[1] for x in list(loadall(data_dir+str(i)+'pickle.dat'))[0]])
  asr_index_raw[1].extend([x[0] for x in list(loadall(data_dir+str(i)+'pickle.dat'))[0]])
  print(f'ASR step {i} done')

#sum_trigrams = np.sum([x for x in asr_index_raw[1]])
asr_index_raw = asr_index_raw[0]
asr_index_raw = [x for x in asr_index_raw if x is not None]
asr_index_raw = list(itertools.chain(*asr_index_raw))
asr_index_raw = np.array(asr_index_raw)
asr_index_raw, asr_counts = np.unique(asr_index_raw, axis=0, return_counts=True)
asr_index_raw = np.insert(asr_index_raw, 3, asr_counts, axis=1)

"""Process Tokens"""
corpTokens = [[],[]]
for i in range(len(lims_list)-1):
  corpTokens_loop = list(itertools.chain(*[x[2] for x in list(loadall(data_dir+str(i)+'pickle.dat'))[0]]))
  corpTokens_loop = np.array(corpTokens_loop)
  corpTokens_loop, token_counts_loop = np.unique(corpTokens_loop, axis=0, return_counts=True)
  corpTokens[0].extend(corpTokens_loop)
  corpTokens[1].extend(token_counts_loop)
  corpTokens_loop = token_counts_loop = []
  print(f'Tokens step {i} done')

corpTokens = np.array(corpTokens)
sum_tokens = np.sum(corpTokens[1])

ASR step 0 done
ASR step 1 done
ASR step 2 done
ASR step 3 done
ASR step 4 done
ASR step 5 done
ASR step 6 done
ASR step 7 done
ASR step 8 done
ASR step 9 done
ASR step 10 done
ASR step 11 done
ASR step 12 done
ASR step 13 done
ASR step 14 done
ASR step 15 done
ASR step 16 done
ASR step 17 done
ASR step 18 done
ASR step 19 done
ASR step 20 done
ASR step 21 done
ASR step 22 done
ASR step 23 done
ASR step 24 done
ASR step 25 done
ASR step 26 done
ASR step 27 done
ASR step 28 done
ASR step 29 done
ASR step 30 done
ASR step 31 done
ASR step 32 done
ASR step 33 done
ASR step 34 done
ASR step 35 done
ASR step 36 done
ASR step 37 done
ASR step 38 done
ASR step 39 done
ASR step 40 done
ASR step 41 done
ASR step 42 done
ASR step 43 done
ASR step 44 done
ASR step 45 done
ASR step 46 done
ASR step 47 done
ASR step 48 done
ASR step 49 done
ASR step 50 done
ASR step 51 done
ASR step 52 done
ASR step 53 done
ASR step 54 done
ASR step 55 done
ASR step 56 done
ASR step 57 done
ASR step 58 done
ASR ste

## Save to CSV

In [ ]:
def row_appender(asr_row, token_idx, total_tokens):
  c_asr = asr_row[3].astype(float)
  # token pos 1
  c_1 = np.sum(token_idx[1][np.where(token_idx[0]==asr_row[0])[0]]).astype(float) 
  # token pos 2
  c_2 = np.sum(token_idx[1][np.where(token_idx[0]==asr_row[1])[0]]).astype(float)
  # token pos 3
  c_3 = np.sum(token_idx[1][np.where(token_idx[0]==asr_row[2])[0]]).astype(float)
  pmi = np.log2(total_tokens**2*c_asr/(c_1*c_2*c_3))

  if asr_row[0] == asr_row[1] and asr_row[0] != asr_row[2]:
    return {'Position 1': asr_row[0], 'Position 2': asr_row[1], 'Position 3': asr_row[2],
            'PMI': pmi, 'ASR Counts': int(c_asr), 'Pos 1 Counts': int(c_1), 'Pos 2 Counts': int(c_2), 'Pos 3 Counts': int(c_3),
            'Pattern': 'AAB', 'Plain Text': ' '.join(tokenizer.convert_ids_to_tokens(asr_row[:3]))}
  elif asr_row[0] == asr_row[2] and asr_row[0] != asr_row[1]:
    return {'Position 1': asr_row[0], 'Position 2': asr_row[1], 'Position 3': asr_row[2],
            'PMI': pmi, 'ASR Counts': int(c_asr), 'Pos 1 Counts': int(c_1), 'Pos 2 Counts': int(c_2), 'Pos 3 Counts': int(c_3),
            'Pattern': 'ABA', 'Plain Text': ' '.join(tokenizer.convert_ids_to_tokens(asr_row[:3]))}
  elif asr_row[1] == asr_row[2] and asr_row[0] != asr_row[1]:
    return {'Position 1': asr_row[0], 'Position 2': asr_row[1], 'Position 3': asr_row[2],
            'PMI': pmi, 'ASR Counts': int(c_asr), 'Pos 1 Counts': int(c_1), 'Pos 2 Counts': int(c_2), 'Pos 3 Counts': int(c_3),
            'Pattern': 'ABB', 'Plain Text': ' '.join(tokenizer.convert_ids_to_tokens(asr_row[:3]))}
  else: print('Pattern ERROR')

asr_df = pd.DataFrame(data={'Position 1': [], 'Position 2': [], 'Position 3': [], 'PMI': [], 'ASR Counts': [], 'Pos 1 Counts': [], 'Pos 2 Counts': [], 'Pos 3 Counts': [], 
                            'Pattern': [],'Plain Text': []})

asr_df = asr_df.astype({'Position 1': 'int', 'Position 2': 'int', 'Position 3': 'int',
                        'ASR Counts': 'int', 'Pos 1 Counts': 'int', 'Pos 2 Counts': 'int', 'Pos 3 Counts': 'int'})

# Parallelize processing
pool = mp.Pool(mp.cpu_count())
asr_df = asr_df.append(pool.starmap(row_appender, [(row, corpTokens, sum_tokens) for row in asr_index_raw[asr_index_raw[:,3] >= 20]]))
pool.close()

asr_df = asr_df.sort_values(by='PMI', ascending=False).reset_index(drop=True)

if '/' in model_name: 
  model_name = model_name.replace('/', '_')

asr_df.to_csv(f'ASR_index_{corpus}_{model_name}_{sum_tokens}Tokens.csv')

# For Google Colab
from google.colab import files
files.download(f'ASR_index_{corpus}_{model_name}_{sum_tokens}Tokens.csv')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>